In [1]:
!pip install boto3
!pip install pickle
!pip install yaml
!pip install sodapy

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import pickle
import boto3

from datetime import date
from sodapy import Socrata

In [2]:
with open('credentials.yaml', 'r') as f:
    try:
        config = yaml.safe_load(f)
    except yaml.YAMLError as exc:
        print(exc)

token = config['api_chicago']['app_token']
username = config['api_chicago']['username']
password = config['api_chicago']['password']
matricula = config['iexe']['matricula']

In [3]:
# code for dataset of food inspections
chicago_dataset = "4ijn-s7e5"

### 1. get_client function

In [4]:
def get_client():
    client = Socrata("data.cityofchicago.org", 
                     token,
                     username=username,
                     password=password)
    
    return client

### 2. Initial intake

Download data from the Food Establishment Inspections API. 

In [5]:
def ingesta_inicial(chicago_dataset, client, limit):
    datasets = client.get(chicago_dataset, limit=limit, offset=0, order='inspection_date')
    
    return datasets

In [6]:
client = get_client()
datasets = ingesta_inicial(chicago_dataset, client, 300000)

In [7]:
len(datasets)

261420

In [8]:
datasets[0]

{'inspection_id': '52234',
 'dba_name': 'Cafe 608',
 'aka_name': 'Cafe 608',
 'license_': '2013328',
 'facility_type': 'Restaurant',
 'risk': 'Risk 1 (High)',
 'address': '608 W BARRY AVE ',
 'city': 'CHICAGO',
 'state': 'IL',
 'zip': '60657',
 'inspection_date': '2010-01-04T00:00:00.000',
 'inspection_type': 'License Re-Inspection',
 'results': 'Pass',
 'latitude': '41.938006880423615',
 'longitude': '-87.6447545707008',
 'location': {'latitude': '41.938006880423615',
  'longitude': '-87.6447545707008'}}

In [9]:
datasets = pd.DataFrame(datasets)

### 3. Initial intake storage

In [21]:
def guardar_ingesta(bucket, bucket_path, dataset):
    session = boto3.Session(
        aws_access_key_id = config['s3']['aws_access_key_id'],
        aws_secret_access_key = config['s3']['aws_secret_access_key'],
        aws_session_token= config['s3']['aws_session_token']
    )

    s3 = session.resource('s3')
    s3.Object(bucket, bucket_path).put(Body=dataset)

### 3a. Bucket creation
##### we create the variable with today's date

In [12]:
TODAY = date.today()
print(TODAY)

2023-10-16


In [22]:
if not 'bucket_creado' in locals():
  print("creando")
  session = boto3.Session(
    aws_access_key_id = config['s3']['aws_access_key_id'],
    aws_secret_access_key = config['s3']['aws_secret_access_key'],
    aws_session_token= config['s3']['aws_session_token']
  )
  s3 = session.resource('s3')
  nombre_bucket = "aplicaciones-cd-1-" + config['iexe']['matricula']
  print(nombre_bucket)
  s3.create_bucket(Bucket=nombre_bucket)
  bucket_creado = True

2023-07-06
creando
aplicaciones-cd-1-mcda22d008


In [11]:
pickle_data = pickle.dumps(datasets)

In [24]:
bucket = "aplicaciones-cd-1-" + config['iexe']['matricula']
key = "ingesta/inicial/inspecciones-historicas-" + str(TODAY) + ".pkl"

guardar_ingesta(bucket, key, pickle_data)

Save the dataset in a pickle in an S3 bucket, the file name dete ends with the day on which the ingestion was performed (obtained dynamically). 

In [13]:
TODAY = date.today()
print(TODAY)
key = "inspecciones-historicas-" + str(TODAY) + ".pkl"
pickle.dump(datasets, open(key,'wb')) 

2023-10-16


### 4. Consecutive intakes

The next time you ingest data you will have to ask specifically from the date from which you have no data.

In [14]:
def ingesta_consecutiva(chicago_dataset, client, fecha, limit):
    new_dataset = client.get(chicago_dataset, limit=limit, where="inspection_date>='{}'".format(fecha))
    
    return new_dataset

In [15]:
client = get_client()

In [16]:
new_dataset = ingesta_consecutiva(chicago_dataset, client, '2020-11-03', 1000)

In [17]:
new_dataset[0]

{'inspection_id': '2454191',
 'dba_name': 'KIRKWOOD BAR & GRILL',
 'aka_name': 'KIRKWOOD BAR & GRILL',
 'license_': '1425',
 'facility_type': 'Restaurant',
 'risk': 'Risk 1 (High)',
 'address': '2934-36 N SHEFFIELD AVE ',
 'city': 'CHICAGO',
 'state': 'IL',
 'zip': '60657',
 'inspection_date': '2020-11-03T00:00:00.000',
 'inspection_type': 'Complaint',
 'results': 'No Entry',
 'latitude': '41.93546443073959',
 'longitude': '-87.65410202519548',
 'location': {'latitude': '41.93546443073959',
  'longitude': '-87.65410202519548'}}

In [18]:
new_dataset = pd.DataFrame(new_dataset)

In [20]:
len(new_dataset)

1000

### 5. Storage of consecutive intakes

In [21]:
pickled_new_data = pickle.dumps(new_dataset)

In [31]:
bucket = "aplicaciones-cd-1-" + config['iexe']['matricula']
key = "ingesta/consecutiva/inspecciones-consecutivas-" + str(TODAY) + ".pkl"

guardar_ingesta(bucket, key, pickled_new_data)

In [22]:
key = "inspecciones-consecutivas-" + str(TODAY) + ".pkl"
pickle.dump(new_dataset, open(key,'wb')) 